In [ ]:
import os
import sys
import datetime
import requests
import pandas as pd
from requests_html import HTML, HTMLSession, AsyncHTMLSession

import time
from selenium import webdriver
import csv
from tqdm import tqdm
from utils import get_comment_post_time

## add MISSING n_participatns and author --> *./de_projekte.csv*

In [ ]:
session = HTMLSession()
    
project_df = pd.read_csv('./de_projekte.csv')
csv_projekte_meta_lines = []
for i, row in project_df.iterrows():   
    
    project_page = session.get(row['href'])
    participants_elem = project_page.html.find('.project-hero__project-size',first=True)
    participants = int(participants_elem.text.split()[0]) if participants_elem else None

    row['participants'] = participants
    csv_projekte_meta_lines.append(row)
#     break

df = pd.DataFrame(csv_projekte_meta_lines)
df.to_csv('./de_projekte.csv', index=None)  

## create --> *./de_blogs_meta.csv*

In [ ]:
session = HTMLSession()

df = pd.read_csv(f'./de_blog_links.csv')

csv_blog_meta_lines = []
for i, row in df.iterrows():    
    
    blog_url = row['blog_href']
    print(blog_url)
    page = session.get(blog_url)
    
#     n of comments
    n_comments_elem = page.html.find('.comment-headline',first=True)
    n_comments = int(n_comments_elem.text.split()[0]) if n_comments_elem else None
    
#     auther and publish_date
    meta_elem = page.html.find('.post-meta',first=True)
    if meta_elem:
        meta_elem = meta_elem.find('p')
    author = meta_elem[0].text if meta_elem else None
    publish_date = str(datetime.datetime.now() - datetime.timedelta(seconds=get_comment_post_time(meta_elem[1].text[4:]))).split('.')[0] if meta_elem else None
    print(publish_date)
    
    
#     participants_elem = page.html.find('.project-hero__project-size',first=True)
#     category_elem = page.html.find('.project-hero__project-category',first=True)
#     timespan_elem = page.html.find('.project-hero__project-runtime',first=True)

#     participants = int(participants_elem.text.split()[0]) if participants_elem else None
#     category = category_elem.text.strip() if category_elem else None
#     timespan = timespan_elem.text.strip() if timespan_elem else None

    content_elem = page.html.find('.post-content__article',first=True)
    content = page.html.find('.post-content__article',first=True).text if content_elem else None
    
    csv_blog_meta_lines.append({
        'project_id' : row['project_id'],
        'blog_id' : row['blog_id'],
        'blog_href' : blog_url,
#         'category' : category,
#         'timespan' : timespan,
#         'n_participants' : participants,
        'author' : author,
        'publish_date' : publish_date,
        'content' : content,
        'n_comments' : n_comments
    })
    
#     break

df = pd.DataFrame(csv_blog_meta_lines)
df.to_csv('./de_blogs_meta.csv', index=None)  

## load --> *./de_projekt_ergebnisse*

In [ ]:
# this should work all the way back... 2017 and earlier

df = pd.read_csv(f'./de_projekte.csv')
session = HTMLSession()

csv_result_line = []
for i, row in df.iterrows():
    
    project_base_url = row['href']
    print(f'\n#{i+1}  {project_base_url}')
    homepage = session.get(project_base_url)
    
    nav_bar = homepage.html.find('.project-nav',first=True)
    if nav_bar:
        print([x.text for x in nav_bar.find('a')])
    if nav_bar and any([x.text == 'Ergebnisse' for x in nav_bar.find('a')]):
        project_results_link = [x for x in nav_bar.find('a') if x.text == 'Ergebnisse'][0].attrs['href']
    else:
        continue
    
    print(f'#{i+1}  {project_results_link}')
    ergebnisse_page = session.get(project_results_link)
    
    # weiterempfehlungsquote
    weiterempfehlungsquote_elem = ergebnisse_page.html.find('.product-conviction__number',first=True)
    weiterempfehlungsquote = int(weiterempfehlungsquote_elem.text[:-1]) if weiterempfehlungsquote_elem else None
    
    spans = ergebnisse_page.html.find('span')
    rating_value = [item for item in spans if item.attrs.get('itemprop') == 'ratingValue']
    review_count = [item for item in spans if item.attrs.get('itemprop') == 'reviewCount']
    rating_value = float(rating_value[0].text) if len(rating_value) == 1 else None
    review_count = int(review_count[0].text.replace('.','')) if len(review_count) == 1 else None
    
    content = homepage.html.find('.post-wrapper',first=True)
    content = content.find('article',first=True).text if content else None
    
    csv_result_line.append({
        'project_id' : row['project_id'],
        'weiterempfehlungsquote (%)' : weiterempfehlungsquote,
        'rating_value' : rating_value,
        'review_count' : review_count,
        'content' : content
    })

df = pd.DataFrame(csv_result_line)
df.to_csv('./de_projekt_ergebnisse.csv', index=None)  